In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'   # OpenMP 라이브러리 충돌 오류 방지
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'    # Tensorflow 로그 메시지(ERROR만 출력)
import linecache
from tqdm import tqdm

import numpy as np
import pandas as pd
import scipy

from statsmodels.tsa.seasonal import seasonal_decompose

import tensorflow as tf

tf.config.set_soft_device_placement(True)

from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.layers import Input, Dense, Conv2D, Conv1D, Conv1DTranspose, Activation, MaxPool2D, MaxPooling1D, AveragePooling1D, BatchNormalization, LayerNormalization, Flatten, Reshape, Conv2DTranspose, LeakyReLU, UpSampling2D, LSTM, Dropout, Add
import tensorflow_addons as tfa
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras import regularizers

from sklearn import metrics
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, roc_curve, auc, precision_recall_curve

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn import tree

from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE, ADASYN, BorderlineSMOTE, SVMSMOTE, KMeansSMOTE, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler


import matplotlib.pyplot as plt
import seaborn as sns
import pywt

import optuna
from optuna.integration import TFKerasPruningCallback

# 라이브러리 버전 확인
print("Numpy 버전:", np.__version__)
print("Pandas 버전:", pd.__version__)
print("Scipy 버전:", scipy.__version__)
print("Tensorflow 버전:", tf.__version__)

Numpy 버전: 1.23.4
Pandas 버전: 2.2.2
Scipy 버전: 1.13.1
Tensorflow 버전: 2.6.0


In [2]:
# GPU 메모리 관리 관련
gpus = tf.config.experimental.list_physical_devices('GPU')

if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)  
    
os.environ["CUDA_VISIBLE_DEVICES"] = "0"    

1 Physical GPUs, 1 Logical GPUs


In [3]:
seed = 2

np.random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)

In [4]:
# Dictionary of scalers
scaler_dict = {
    'MinMaxScaler': MinMaxScaler(),
    'StandardScaler': StandardScaler(),
    'RobustScaler': RobustScaler()
}

# Dictionary of samplers
sampler_dict = {
    'SMOTE': SMOTE(),
    'ADASYN': ADASYN(),
    'KMeansSMOTE': KMeansSMOTE(),
    'BorderlineSMOTE': BorderlineSMOTE(),
    'SVMSMOTE': SVMSMOTE()
}

In [5]:
# Load labels
train_cat = pd.read_pickle(r'C:\Users\MINJEIP-LAB\Dropbox\KAMP 2024_4th\data\dataset\train_categorical.pkl')
val_cat = pd.read_pickle(r'C:\Users\MINJEIP-LAB\Dropbox\KAMP 2024_4th\data\dataset\val_categorical.pkl')
test_cat = pd.read_pickle(r'C:\Users\MINJEIP-LAB\Dropbox\KAMP 2024_4th\data\dataset\test_categorical.pkl')

train_num = pd.read_pickle(r'C:\Users\MINJEIP-LAB\Dropbox\KAMP 2024_4th\data\dataset\train_numerical.pkl')
val_num = pd.read_pickle(r'C:\Users\MINJEIP-LAB\Dropbox\KAMP 2024_4th\data\dataset\val_numerical.pkl')
test_num = pd.read_pickle(r'C:\Users\MINJEIP-LAB\Dropbox\KAMP 2024_4th\data\dataset\test_numerical.pkl')

y_train = pd.read_pickle(r'C:\Users\MINJEIP-LAB\Dropbox\KAMP 2024_4th\data\dataset\train_label.pkl')
y_val = pd.read_pickle(r'C:\Users\MINJEIP-LAB\Dropbox\KAMP 2024_4th\data\dataset\val_label.pkl')
y_test = pd.read_pickle(r'C:\Users\MINJEIP-LAB\Dropbox\KAMP 2024_4th\data\dataset\test_label.pkl')

In [6]:
# # Updated preprocess_data function with conversion to NumPy arrays
# def preprocess_data(scaler_key='MinMaxScaler', sampler_key='None'):
#     # Scaling numerical data
#     if scaler_key != 'None':
#         scaler = scaler_dict[scaler_key]
#         train_numerical_scaled = pd.DataFrame(scaler.fit_transform(train_num), columns=train_num.columns)
#         valid_numerical_scaled = pd.DataFrame(scaler.transform(valid_num), columns=valid_num.columns)
#         test_numerical_scaled = pd.DataFrame(scaler.transform(test_num), columns=test_num.columns)
#     else:
#         train_numerical_scaled = train_num
#         valid_numerical_scaled = valid_num
#         test_numerical_scaled = test_num

#     # Concatenate scaled numerical and categorical data
#     X_train = pd.concat([train_cat.reset_index(drop=True), train_numerical_scaled], axis=1)
#     X_valid = pd.concat([valid_cat.reset_index(drop=True), valid_numerical_scaled], axis=1)
#     X_test = pd.concat([test_cat.reset_index(drop=True), test_numerical_scaled], axis=1)    
    
#     # Convert DataFrames to NumPy arrays
#     X_train = X_train.values
#     X_valid = X_valid.values
#     X_test = X_test.values

#     # Sampling
#     if sampler_key != 'None':
#         sampler = sampler_dict[sampler_key]
#         X_train_resampled, y_train_resampled = sampler.fit_resample(X_train, y_train)
#     else:
#         X_train_resampled = X_train
#         y_train_resampled = y_train
    
#     return X_train_resampled, X_valid, X_test, y_train_resampled

In [7]:
import optuna
from optuna.integration import TFKerasPruningCallback
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, MaxPooling1D, Flatten, Dense, Dropout, Add
from tensorflow.keras.models import Model
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import numpy as np
import os
from imblearn.over_sampling import ADASYN, SMOTE, RandomOverSampler, BorderlineSMOTE, SVMSMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler

class Optimize1DResNet():
    def __init__(self,
                 train_numerical,
                 train_categorical,
                 train_label,
                 val_numerical,
                 val_categorical,
                 val_label,
                 test_numerical,
                 test_categorical,
                 test_label):
        
        self.train_numerical = train_numerical.reset_index(drop=True)
        self.train_categorical = train_categorical.reset_index(drop=True)
        self.train_label = train_label.reset_index(drop=True)

        self.val_numerical = val_numerical.reset_index(drop=True)
        self.val_categorical = val_categorical.reset_index(drop=True)
        self.val_label = val_label.reset_index(drop=True)

        self.test_numerical = test_numerical.reset_index(drop=True)
        self.test_categorical = test_categorical.reset_index(drop=True)
        self.test_label = test_label.reset_index(drop=True)

        self.best_score = 0
        self.scaler = None

        os.makedirs('./model/', exist_ok=True)

    def objective(self, trial):
        filters_1 = trial.suggest_int('filters_1', 3, 4)
        filters_2 = trial.suggest_int('filters_2', 3, 5)
        filters_3 = trial.suggest_int('filters_3', 3, 6)
        dense_units_1 = trial.suggest_int('dense_units_1', 4, 6)
        dense_units_2 = trial.suggest_int('dense_units_2', 3, 5)
        dropout_rate = trial.suggest_float('dropout_rate', 0, 0.5)
        learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True)
        threshold = trial.suggest_float('threshold', 0.2, 0.8)
        sampler_key = trial.suggest_categorical('sampler', ['ADASYN', 'SMOTE', 'RandomUnderSampler', 'RandomOverSampler', 'BorderlineSMOTE', 'SVMSMOTE', 'None'])
        scaler_key = trial.suggest_categorical('scaler', ['MinMaxScaler', 'StandardScaler', 'RobustScaler', 'None'])
                

        sampler_dict = {
            'ADASYN': ADASYN(random_state=25),
            'SMOTE': SMOTE(random_state=25),
            'RandomUnderSampler': RandomUnderSampler(random_state=25),
            'RandomOverSampler': RandomOverSampler(random_state=25),
            'BorderlineSMOTE': BorderlineSMOTE(random_state=25),
            'SVMSMOTE': SVMSMOTE(random_state=25),
        }

        scaler_dict = {
            'MinMaxScaler': MinMaxScaler(),
            'StandardScaler': StandardScaler(),
            'RobustScaler': RobustScaler(),
        }

        y_train = self.train_label
        y_val = self.val_label
        y_test = self.test_label

        if scaler_key == 'None':
            X_train = pd.concat([self.train_numerical, self.train_categorical], axis=1)
            X_val = pd.concat([self.val_numerical, self.val_categorical], axis=1)
            X_test = pd.concat([self.test_numerical, self.test_categorical], axis=1)

            scaler = None

        else:
            scaler = scaler_dict[scaler_key]
            train_numerical = pd.DataFrame(scaler.fit_transform(self.train_numerical), columns=self.train_numerical.columns)
            val_numerical = pd.DataFrame(scaler.transform(self.val_numerical), columns=self.train_numerical.columns)
            test_numerical = pd.DataFrame(scaler.transform(self.test_numerical), columns=self.train_numerical.columns)

            X_train = pd.concat([train_numerical, self.train_categorical], axis=1)
            X_val = pd.concat([val_numerical, self.val_categorical], axis=1)
            X_test = pd.concat([test_numerical, self.test_categorical], axis=1)

        if sampler_key == 'None':
            x_resampled, y_resampled = X_train, y_train

        else:
            sampler = sampler_dict[sampler_key]
            print('Start resampling...')
            x_resampled, y_resampled = sampler.fit_resample(X_train, y_train)
            print('Resampling finished...')

        # 입력 데이터를 Conv1D에 맞게 3차원으로 변환
        x_resampled = np.array(x_resampled).reshape((x_resampled.shape[0], x_resampled.shape[1], 1))
        X_val = np.array(X_val).reshape((X_val.shape[0], X_val.shape[1], 1))
        X_test = np.array(X_test).reshape((X_test.shape[0], X_test.shape[1], 1))

        x_resampled = tf.convert_to_tensor(x_resampled, dtype=tf.float32)
        X_val = tf.convert_to_tensor(X_val, dtype=tf.float32)
        X_test = tf.convert_to_tensor(X_test, dtype=tf.float32)
        y_resampled = tf.convert_to_tensor(y_resampled, dtype=tf.float32)
        y_val = tf.convert_to_tensor(y_val, dtype=tf.float32)
        y_test = tf.convert_to_tensor(y_test, dtype=tf.float32)
        
        # Build the model
        model = self.build_1d_resnet_classifier(input_dim=x_resampled.shape[1],
                                                filters_1=2**filters_1,
                                                filters_2=2**filters_2,
                                                filters_3=2**filters_3,
                                                dense_units_1=2**dense_units_1,
                                                dense_units_2=2**dense_units_2,
                                                dropout_rate=dropout_rate)

        # Compile the model
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                      loss='binary_crossentropy',
                      metrics=['accuracy'])
        
        early_stopping = tf.keras.callbacks.EarlyStopping(patience=30)
        rlr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=10, verbose=1)
        checkpoint = tf.keras.callbacks.ModelCheckpoint("best_model_1dResNet/best_model_1dResNet.h5", monitor="val_loss", save_best_only=True, save_format="h5", verbose=0)

        callbacks = [early_stopping, rlr, checkpoint, TFKerasPruningCallback(trial, 'val_loss')]

        # Train the model
        model.fit(x_resampled, y_resampled, epochs=1000, batch_size=512, callbacks=callbacks, validation_data=(X_val, y_val))

        # Evaluate the model
        preds = model.predict(X_test)
        preds_binary = (preds > threshold).astype(int)
        f1 = f1_score(y_test, preds_binary)

        self.best_score = max(f1, self.best_score)

        if self.best_score == f1:
            model.save(f"./model/best_model_{f1:.4f}.h5")
            self.scaler = scaler

        return f1

    def build_1d_resnet_classifier(self, input_dim, filters_1, filters_2, filters_3, dense_units_1, dense_units_2, dropout_rate):
        inputs = Input(shape=(input_dim, 1))
        
        # 초기 Conv Layer
        x = Conv1D(filters_1, 3, padding="same")(inputs)
        x = BatchNormalization()(x)
        x = tfa.layers.GELU()(x)

        # Residual Block 1
        x = self.residual_block(x, filters=filters_1)
        
        # Residual Block 2
        x = self.residual_block(x, filters=filters_2)

        # Residual Block 3
        x = self.residual_block(x, filters=filters_3)

        # Flatten Layer
        x = Flatten()(x)
        
        # Fully Connected Layers
        x = Dense(dense_units_1)(x)
        x = tfa.layers.GELU()(x)
        
        x = Dense(dense_units_2)(x)
        x = tfa.layers.GELU()(x)
        x = Dropout(dropout_rate)(x)
        
        # 출력 Layer
        outputs = Dense(1, activation='sigmoid')(x)
        
        model = Model(inputs=inputs, outputs=outputs)
        return model

    def residual_block(self, x, filters, kernel_size=3, padding='same', use_pooling=True):
        shortcut = x

        # 입력 차원이 맞지 않는 경우 맞추기 위해 1D 컨볼루션 적용
        if x.shape[-1] != filters:
            shortcut = Conv1D(filters, kernel_size=1, padding=padding)(shortcut)

        # 첫 번째 Conv Layer
        x = Conv1D(filters, kernel_size, padding=padding)(x)
        x = BatchNormalization()(x)
        x = tfa.layers.GELU()(x)

        # 두 번째 Conv Layer
        x = Conv1D(filters, kernel_size, padding=padding)(x)
        x = BatchNormalization()(x)

        # 스킵 연결 (identity mapping)
        x = Add()([x, shortcut])
        x = tfa.layers.GELU()(x)

        # 필요 시 MaxPooling 적용
        if use_pooling:
            x = MaxPooling1D(pool_size=2)(x)

        return x

    def optimize(self, study_name='KAMP2024', direction='maximize', storage='sqlite:////mnt/d/OPTIMIZE_1D_RESNET.db', load_if_exists=True, n_trials=1500):
        study = optuna.create_study(
            study_name=study_name,
            direction=direction,
            storage=storage,
            load_if_exists=load_if_exists
        )

        study.optimize(self.objective, n_trials=n_trials)


In [8]:
optimizer = Optimize1DResNet(
    train_num,
    train_cat,
    y_train,
    val_num,
    val_cat,
    y_val,
    test_num,
    test_cat,
    y_test,
)

In [9]:
optimizer.optimize(storage='sqlite:///C:/Users/MINJEIP-LAB/Dropbox/KAMP 2024_4th/code/mj/OPTIMIZE_1D_RESNET.db')

[I 2024-11-06 07:32:54,586] A new study created in RDB with name: KAMP2024


Start resampling...
Resampling finished...
Epoch 1/1000
271/271 [==============================] - 8s 10ms/step - loss: 0.3624 - accuracy: 0.8471 - val_loss: 0.2903 - val_accuracy: 0.9788
Epoch 2/1000
 16/271 [>.............................] - ETA: 1s - loss: 0.2205 - accuracy: 0.9246

c:\Users\MINJEIP-LAB\anaconda3\envs\env_smartplug5\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


271/271 [==============================] - 2s 7ms/step - loss: 0.2023 - accuracy: 0.9315 - val_loss: 0.0933 - val_accuracy: 0.9769
Epoch 3/1000
271/271 [==============================] - 2s 7ms/step - loss: 0.1761 - accuracy: 0.9391 - val_loss: 0.1255 - val_accuracy: 0.9534
Epoch 4/1000
271/271 [==============================] - 2s 7ms/step - loss: 0.1578 - accuracy: 0.9452 - val_loss: 0.0817 - val_accuracy: 0.9702
Epoch 5/1000
271/271 [==============================] - 2s 7ms/step - loss: 0.1486 - accuracy: 0.9482 - val_loss: 0.0810 - val_accuracy: 0.9688
Epoch 6/1000
271/271 [==============================] - 2s 7ms/step - loss: 0.1371 - accuracy: 0.9510 - val_loss: 0.0800 - val_accuracy: 0.9682
Epoch 7/1000
271/271 [==============================] - 2s 7ms/step - loss: 0.1289 - accuracy: 0.9542 - val_loss: 0.2033 - val_accuracy: 0.9135
Epoch 8/1000
271/271 [==============================] - 2s 7ms/step - loss: 0.1198 - accuracy: 0.9574 - val_loss: 0.1114 - val_accuracy: 0.9532
Epoch

c:\Users\MINJEIP-LAB\anaconda3\envs\env_smartplug5\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
[I 2024-11-06 07:35:12,734] Trial 0 finished with value: 0.9576783555018138 and parameters: {'filters_1': 3, 'filters_2': 5, 'filters_3': 6, 'dense_units_1': 4, 'dense_units_2': 3, 'dropout_rate': 0.29335962491139633, 'learning_rate': 0.00042650277257919115, 'threshold': 0.6533654803086162, 'sampler': 'SMOTE', 'scaler': 'None'}. Best is trial 0 with value: 0.9576783555018138.


...

Start resampling...
Resampling finished...
Epoch 1/1000
271/271 [==============================] - 4s 8ms/step - loss: 0.3151 - accuracy: 0.8534 - val_loss: 11.0543 - val_accuracy: 0.0477
Epoch 2/1000
 17/271 [>.............................] - ETA: 1s - loss: 0.1069 - accuracy: 0.9638

c:\Users\MINJEIP-LAB\anaconda3\envs\env_smartplug5\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


271/271 [==============================] - 2s 7ms/step - loss: 0.0701 - accuracy: 0.9760 - val_loss: 0.4599 - val_accuracy: 0.9575
Epoch 3/1000
271/271 [==============================] - 2s 7ms/step - loss: 0.0462 - accuracy: 0.9850 - val_loss: 0.4905 - val_accuracy: 0.9624
Epoch 4/1000
271/271 [==============================] - 2s 7ms/step - loss: 0.0337 - accuracy: 0.9887 - val_loss: 11.9002 - val_accuracy: 0.0631
Epoch 5/1000
271/271 [==============================] - 2s 7ms/step - loss: 0.0285 - accuracy: 0.9905 - val_loss: 0.4510 - val_accuracy: 0.9589
Epoch 6/1000
271/271 [==============================] - 2s 7ms/step - loss: 0.0250 - accuracy: 0.9913 - val_loss: 92.6666 - val_accuracy: 0.0427
Epoch 7/1000
271/271 [==============================] - 2s 7ms/step - loss: 0.0211 - accuracy: 0.9929 - val_loss: 54.0671 - val_accuracy: 0.0427
Epoch 8/1000
271/271 [==============================] - 2s 8ms/step - loss: 0.0391 - accuracy: 0.9875 - val_loss: 46.4646 - val_accuracy: 0.0427
E

[W 2024-11-06 12:19:41,747] Trial 707 failed with parameters: {'filters_1': 3, 'filters_2': 5, 'filters_3': 6, 'dense_units_1': 6, 'dense_units_2': 4, 'dropout_rate': 0.3440957260736134, 'learning_rate': 0.0045965767876714955, 'threshold': 0.5397514785367445, 'sampler': 'SMOTE', 'scaler': 'RobustScaler'} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\MINJEIP-LAB\anaconda3\envs\env_smartplug5\lib\site-packages\optuna\study\_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\MINJEIP-LAB\AppData\Local\Temp\ipykernel_42348\2025580834.py", line 135, in objective
    model.fit(x_resampled, y_resampled, epochs=1000, batch_size=512, callbacks=callbacks, validation_data=(X_val, y_val))
  File "c:\Users\MINJEIP-LAB\anaconda3\envs\env_smartplug5\lib\site-packages\keras\engine\training.py", line 1184, in fit
    tmp_logs = self.train_function(iterator)
  File "c:\Users\MINJEIP-LAB\anaconda3\envs\env_sma

KeyboardInterrupt: 

In [ ]:
# Define the objective function for Optuna
def objective(trial):
    # Suggest hyperparameters
    conv_filters_1 = trial.suggest_int('conv_filters_1', 3, 6)
    conv_filters_2 = trial.suggest_int('conv_filters_2', 3, 7)
    conv_filters_3 = trial.suggest_int('conv_filters_3', 4, 8)
    dense_units_1 = trial.suggest_int('dense_units_1', 4, 7)
    dense_units_2 = trial.suggest_int('dense_units_2', 3, 6)
    dropout_rate = trial.suggest_float('dropout_rate', 0, 0.5)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True)

    # Build the model
    model = build_1d_conv_classifier(input_dim=X_train.shape[1],
                                     conv_filters_1=2**conv_filters_1,
                                     conv_filters_2=2**conv_filters_2,
                                     conv_filters_3=2**conv_filters_3,
                                     dense_units_1=2**dense_units_1,
                                     dense_units_2=2**dense_units_2,
                                     dropout_rate=dropout_rate)

    # Compile the model
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='binary_crossentropy',
                  metrics='binary_crossentropy')

# epochs 및 batch_size 결정
epochs = 1000
batch_size = 512

patience = 30

# 콜백 설정
early_stopping = EarlyStopping(patience=patience)
rlr = ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=10, verbose=1)
checkpoint = ModelCheckpoint("best_model_1dConv/best_model_1dConv.h5", monitor="val_loss", save_best_only=True, save_format="h5", verbose=0)

callbacks = [early_stopping, rlr, checkpoint]

# 모델 학습
history = model.fit(
    X_train,
    y_train,
    epochs=epochs,
    batch_size=batch_size,
    callbacks=callbacks,
    validation_data=(X_valid, y_valid)
)



    # Train the model
    model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_valid, y_valid), verbose=0)

    # Evaluate the model
    preds = model.predict(X_valid)
    preds_binary = (preds > 0.5).astype(int)
    f1 = f1_score(y_valid, preds_binary)

    return f1

# Modified function to build the Conv1D model
def build_1d_conv_classifier(input_dim, conv_filters_1, conv_filters_2, conv_filters_3, dense_units_1, dense_units_2, dropout_rate):
    inputs = Input(shape=(input_dim, 1))
    
    # Conv Layer 1
    x = Conv1D(conv_filters_1, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = tfa.layers.GELU()(x)
    x = MaxPooling1D(pool_size=2)(x)
    
    # Conv Layer 2
    x = Conv1D(conv_filters_2, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = tfa.layers.GELU()(x)
    x = MaxPooling1D(pool_size=2)(x)
    
    # Conv Layer 3
    x = Conv1D(conv_filters_3, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = tfa.layers.GELU()(x)
    x = MaxPooling1D(pool_size=2)(x)
    
    # Flatten Layer
    x = Flatten()(x)
    
    # Fully Connected Layers
    x = Dense(dense_units_1)(x)
    x = tfa.layers.GELU()(x)
    
    x = Dense(dense_units_2)(x)
    x = tfa.layers.GELU()(x)
    x = Dropout(dropout_rate)(x)
    
    # Output Layer
    outputs = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    return model

# Load your dataset
# Assuming you have a dataset with X and y
# X, y = load_your_data()
X = np.random.rand(1000, 100)  # Example dataset
y = np.random.randint(0, 2, 1000)

# Split the dataset into training, validation, and test sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# Run Optuna optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

# Display best trial
print("Best trial:")
trial = study.best_trial
print(f"  F1 Score: {trial.value}")
print("  Best hyperparameters: ", trial.params)

# Train final model with best parameters
best_params = trial.params
final_model = build_1d_conv_classifier(input_dim=X_train.shape[1],
                                       conv_filters_1=best_params['conv_filters_1'],
                                       conv_filters_2=best_params['conv_filters_2'],
                                       conv_filters_3=best_params['conv_filters_3'],
                                       dense_units_1=best_params['dense_units_1'],
                                       dense_units_2=best_params['dense_units_2'],
                                       dropout_rate=best_params['dropout_rate'])

final_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=best_params['learning_rate']),
                    loss='binary_crossentropy',
                    metrics=['accuracy'])

final_model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_valid, y_valid))


In [ ]:
X_train, X_valid, X_test, y_train = preprocess_data(scaler_key='MinMaxScaler', sampler_key='None')

In [ ]:
y_train = np.array(y_train)
y_valid = np.array(y_valid)
y_test = np.array(y_test)

In [ ]:
# #from eli5.sklearn import PermutationImportance
# import scipy
# from sklearn.pipeline import Pipeline
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.metrics import f1_score, confusion_matrix
# from sklearn.model_selection import train_test_split
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
# from lightgbm import LGBMClassifier, plot_importance
# from xgboost import XGBClassifier
# from sklearn.model_selection import StratifiedShuffleSplit
# import warnings
# warnings.filterwarnings('ignore')

# t_test = []
# for i, col in enumerate(df_encoded.columns):
#  t=scipy.stats.ttest_ind(df_encoded[df_encoded['passorfail']==1][col], df_encoded[df_encoded['passorfail']==0][col],
# equal_var=False)
#  t_test.append([col, t[0], t[1]])
# df_ttest = pd.DataFrame(t_test, columns=['col', 'tvalue', 'pvalue'])
# use_t_list = []
# for i in range(len(df_ttest)):
#     if df_ttest['pvalue'][i] < 0.1:
#         use_t_list.append(df_ttest['col'][i])
# df_ttest

In [ ]:
# X = df_encoded[use_t_list].drop('passorfail', axis=1).values
# y = df_encoded['passorfail'].values

In [ ]:
# # 60% train, 20% validation, 20% test로 분할
# X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
# X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# # MinMax 스케일링 적용
# scaler = MinMaxScaler()

# # Train set으로 스케일러 학습 및 변환
# X_train = scaler.fit_transform(X_train)

# # Validation과 Test set은 학습된 스케일러로 변환만 적용
# X_valid = scaler.transform(X_valid)
# X_test = scaler.transform(X_test)

In [ ]:
# 입력 데이터를 Conv1D에 맞게 3차원으로 변환
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_valid = X_valid.reshape((X_valid.shape[0], X_valid.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

In [ ]:
X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
X_valid = tf.convert_to_tensor(X_valid, dtype=tf.float32)
X_test = tf.convert_to_tensor(X_test, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)
y_valid = tf.convert_to_tensor(y_valid, dtype=tf.float32)
y_test = tf.convert_to_tensor(y_test, dtype=tf.float32)

In [ ]:
# def residual_block(x, filters, kernel_size=3, padding='same', use_pooling=True):
#     shortcut = x

#     # 입력 차원이 맞지 않는 경우 맞추기 위해 1D 컨볼루션 적용
#     if x.shape[-1] != filters:
#         shortcut = Conv1D(filters, kernel_size=1, padding=padding)(shortcut)

#     # 첫 번째 Conv Layer
#     x = Conv1D(filters, kernel_size, padding=padding)(x)
#     x = BatchNormalization()(x)
#     x = tfa.layers.GELU()(x)

#     # 두 번째 Conv Layer
#     x = Conv1D(filters, kernel_size, padding=padding)(x)
#     x = BatchNormalization()(x)

#     # 스킵 연결 (identity mapping)
#     x = Add()([x, shortcut])
#     x = tfa.layers.GELU()(x)

#     # 필요 시 MaxPooling 적용
#     if use_pooling:
#         x = MaxPooling1D(pool_size=2)(x)

#     return x

# def build_1d_resnet_classifier(input_dim):
#     inputs = Input(shape=(input_dim, 1))
    
#     # 초기 Conv Layer
#     x = Conv1D(32, 3, padding="same")(inputs)
#     x = BatchNormalization()(x)
#     x = tfa.layers.GELU()(x)

#     # Residual Block 1
#     x = residual_block(x, filters=16)
    
#     # Residual Block 2
#     x = residual_block(x, filters=32)

#     # Residual Block 3
#     x = residual_block(x, filters=64)

#     # Flatten Layer
#     x = Flatten()(x)
    
#     # Fully Connected Layer
#     x = Dense(64)(x)
#     x = tfa.layers.GELU()(x)
    
#     x = Dense(16)(x)
#     x = tfa.layers.GELU()(x)
#     x = Dropout(0.2)(x)
    
#     # 출력 Layer
#     outputs = Dense(1, activation='sigmoid')(x)
    
#     model = Model(inputs=inputs, outputs=outputs)
#     return model

# model = build_1d_resnet_classifier(input_dim=100)
# model.summary()


In [ ]:
def build_1d_conv_classifier(input_dim):
    inputs = Input(shape=(input_dim, 1))
    
    # Conv Layer 1
    x = Conv1D(16, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = tfa.layers.GELU()(x)
    x = MaxPooling1D(pool_size=2)(x)
    # x = tfa.layers.AdaptiveMaxPooling1D(output_size=13)(x)
    # x = Dropout(0.1)(x)
    
    # Conv Layer 2
    x = Conv1D(32, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = tfa.layers.GELU()(x)
    x = MaxPooling1D(pool_size=2)(x)
    # x = Dropout(0.1)(x)
    
    # Conv Layer 3
    x = Conv1D(64, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = tfa.layers.GELU()(x)
    x = MaxPooling1D(pool_size=2)(x)
    # x = tfa.layers.AdaptiveMaxPooling1D(output_size=3)(x)
    # x = Dropout(0.1)(x)
    
    # # Conv Layer 4
    # x = Conv1D(128, 3, padding="same")(x)
    # x = BatchNormalization()(x)
    # x = tfa.layers.GELU()(x)
    # x = MaxPooling1D(pool_size=2)(x)
    # # x = Dropout(0.1)(x)
    
    # Flatten Layer
    x = Flatten()(x)
    
    # Fully Connected Layer
    x = Dense(64)(x)
    x = tfa.layers.GELU()(x)
    # x = Dropout(0.1)(x)
    
    x = Dense(16)(x)
    x = tfa.layers.GELU()(x)
    x = Dropout(0.2)(x)
    
    # Output Layer
    outputs = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
# def build_1d_conv_classifier(input_dim):
#     inputs = Input(shape=(input_dim, 1))
    
#     # Conv Layer 1
#     x = Conv1D(16, 3, padding="same")(inputs)
#     x = LayerNormalization()(x)
#     x = tfa.layers.GELU()(x)
#     x = MaxPooling1D(pool_size=2)(x)
#     # x = Dropout(0.1)(x)
    
#     # Conv Layer 2
#     x = Conv1D(32, 3, padding="same")(x)
#     x = LayerNormalization()(x)
#     x = tfa.layers.GELU()(x)
#     x = MaxPooling1D(pool_size=2)(x)
#     # x = Dropout(0.1)(x)
    
#     # Conv Layer 3
#     x = Conv1D(64, 3, padding="same")(x)
#     x = LayerNormalization()(x)
#     x = tfa.layers.GELU()(x)
#     x = MaxPooling1D(pool_size=2)(x)
#     # x = Dropout(0.1)(x)
    
#     # # Conv Layer 4
#     # x = Conv1D(128, 3, padding="same")(x)
#     # x = BatchNormalization()(x)
#     # x = tfa.layers.GELU()(x)
#     # x = MaxPooling1D(pool_size=2)(x)
#     # # x = Dropout(0.1)(x)
    
#     # Flatten Layer
#     x = Flatten()(x)
    
#     # Fully Connected Layer
#     x = Dense(32)(x)
#     x = tfa.layers.GELU()(x)
#     # x = Dropout(0.1)(x)
    
#     x = Dense(8)(x)
#     x = tfa.layers.GELU()(x)
#     x = Dropout(0.1)(x)
    
#     # Output Layer
#     outputs = Dense(1, activation='sigmoid')(x)
    
#     model = Model(inputs=inputs, outputs=outputs)
#     return model

In [ ]:
# 모델 컴파일
input_dim = X_train.shape[1]
model = build_1d_conv_classifier(input_dim)
# model = build_1d_resnet_classifier(input_dim)

# model = LSTM_AE(train_x)
# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001, clipnorm=1.0), loss='binary_crossentropy', metrics=tf.keras.metrics.AUC(name='auc'))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001), loss='binary_crossentropy', metrics='binary_crossentropy')
model.summary()

In [ ]:
# epochs 및 batch_size 결정
epochs = 1000
batch_size = 512

patience = 30

# 콜백 설정
early_stopping = EarlyStopping(patience=patience)
rlr = ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=10, verbose=1)
checkpoint = ModelCheckpoint("best_model_1dConv/best_model_1dConv.h5", monitor="val_loss", save_best_only=True, save_format="h5", verbose=0)

callbacks = [early_stopping, rlr, checkpoint]

# 모델 학습
history = model.fit(
    X_train,
    y_train,
    epochs=epochs,
    batch_size=batch_size,
    callbacks=callbacks,
    validation_data=(X_valid, y_valid)
)

In [ ]:
# Model loss
fig, ax = plt.subplots(figsize=(10,5))#, dpi=300)

plt.figure(figsize=(8, 6))#, dpi=300)  # 해상도 조절
ax.plot(history.history['loss'], 'g', label='Train', linewidth=1)
ax.plot(history.history['val_loss'], 'r', label='Validation', linewidth=1)
ax.set_title('Model loss', fontsize=16)
ax.set_ylabel('Loss(mse)')
ax.set_xlabel('Epoch')
ax.legend(loc='upper right')
plt.show()

In [ ]:
y_pred_prob = model.predict(X_test).ravel()
y_pred = (y_pred_prob > 0.5).astype(int)

In [ ]:
# F1 Score
test_val_f1 = f1_score(y_test, y_pred)
print(f"Test F1 Score: {test_val_f1}")

# Accuracy
test_val_accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {test_val_accuracy}")

# Confusion Matrix
test_conf_matrix = confusion_matrix(y_test, y_pred)
print(f"Confusion Matrix:\n{test_conf_matrix}")

# ROC Curve
fpr, tpr, _ = roc_curve(y_test, y_pred_prob)

In [ ]:
# ROC Curve
fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
roc_auc = auc(fpr, tpr)
plt.figure(figsize=(8, 6), dpi=300)  # 해상도 조절
plt.title('ROC')
plt.plot(fpr, tpr, label='AUC = %0.4f'%roc_auc, linewidth=2, color='b')
plt.legend(loc='lower right')
plt.plot([0,1], [0,1], 'r--')
plt.xlim([-0.001, 1])
plt.ylim([0, 1.001])
plt.ylabel('TPR')
plt.xlabel('FPR')
plt.grid(True, linestyle='--', alpha=0.6)  # 그리드 추가
plt.show()

In [ ]:
# Precision-Recall Curve
precision, recall, _ = precision_recall_curve(y_test, y_pred_prob)
plt.figure(figsize=(8, 6), dpi=300)  # 해상도 조절
plt.plot(recall, precision, color='g', lw=2, label='Precision-Recall Curve')
plt.xlabel('Recall', fontsize=12)
plt.ylabel('Precision', fontsize=12)
plt.title('Precision-Recall Curve', fontsize=14)
plt.legend(loc='lower left', fontsize=10)  # 범례 위치 및 글꼴 크기 조정
plt.grid(True, linestyle='--', alpha=0.6)  # 그리드 추가
plt.xlim(0, 1)
plt.ylim(0, 1)
